In [1]:
import torch
import torch.distributions as dist
import torch.nn as nn
import torch.nn.functional as Fcn
import torch.optim as optim

In [32]:
# Creating Ising spins and Calculating Hamiltonian of the Ising system
class Ising():
    
    def __init__(self, nRow, nCol):
        self.spins = torch.zeros(nRow, nCol)
        self.probs = torch.rand(nRow, nCol)
        for i in range(nRow):
            for j in range(nCol):
                if self.probs[i][j] < 0.5:
                    self.spins[i][j] = 1
                else:
                    self.spins[i][j] = -1
    
    def Hamiltonian(self):
        H = 0.
        J = 1.
        nRow = self.spins.size()[0]
        nCol = self.spins.size()[1]
        for i in range(nRow):
            for j in range(nCol):
                if i < 1:
                    H -= J * self.spins[i][j] * self.spins[i+1][j]
                elif i > nRow - 2:
                    H -= J * self.spins[i][j] * self.spins[i-1][j]
                else:
                    H -= J * self.spins[i][j] * self.spins[i+1][j]
                    H -= J * self.spins[i][j] * self.spins[i-1][j]
                
                if j < 1:
                    H -= J * self.spins[i][j] * self.spins[i][j+1]
                elif j > nCol - 2:
                    H -= J * self.spins[i][j] * self.spins[i][j-1]
                else:
                    H -= J * self.spins[i][j] * self.spins[i][j+1]
                    H -= J * self.spins[i][j] * self.spins[i][j-1]
        return H/2   #to avoid double count

In [33]:
# Fuction to create a training data set by using Metropolis algorithm
def Data_for_train(data_size, nRow, nCol):

    for i in range(data_size):
    
        ising = Ising(nRow, nCol)
        H_new = ising.Hamiltonian()
    
        # Reshape of a matrix of Ising spins to a vector as the visible layer
        spin = ising.spins.view(nRow*nCol)
        v = (1 - spin)/2   # spin 1 --> 0 ,   spin -1 --> 1
    
        # save visible layers as row vectors of the training data matrix
        if i == 0:
            data = v.unsqueeze(0)
        else:
            if H_new <= H:
                data = torch.cat((data, v.unsqueeze(0)), dim = 0)
            else:
                B_sample = torch.bernoulli(torch.exp(H - H_new))
                if B_sample == 1:
                    data = torch.cat((data, v.unsqueeze(0)), dim = 0)
                else:
                    data = torch.cat((data, data[i-1].unsqueeze(0)), dim = 0)
        H = H_new
        
    return data

In [48]:
# Creating the RBM Architecture (weights, biases)
class RBM():
    
    # Initiate RBM parameters
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(nh)
        self.b = torch.randn(nv)
    
    def Energy(self, v, h): 
        ah = torch.dot(self.a, h)
        bv = torch.dot(self.b, v)
        hWv = torch.dot(h, torch.mv(self.W, v))
        E = - ah - bv - hWv
        return E
        
    # Contrastive Divergence
    def update_CD(self, Batch, learning_rate):
        
        batch_size = Batch.size()[0]
        delta_W = torch.zeros_like(self.W)
        delta_a = torch.zeros_like(self.a)
        delta_b = torch.zeros_like(self.b)
        for i in range(batch_size):
            a = self.a
            b = self.b
            num_iterations = 1
            
            # k-iterations of Gibbs sampling
            for k in range(num_iterations):
                if k == 0:
                    v_0 = Batch[i]
                    Wv = torch.mv(self.W, v_0)
                    p_h_given_v_0 = torch.sigmoid(a + Wv)
                    h_0 = torch.bernoulli(p_h_given_v_0)
                    h_old = h_0
                else:
                    h_old = h_new
                
                hW = torch.mv(torch.t(self.W), h_old)
                p_v_given_h = torch.sigmoid(b + hW)
                v_new = torch.bernoulli(p_v_given_h)
                Wv = torch.mv(self.W, v_new)
                p_h_given_v = torch.sigmoid(a + Wv)
                h_new = torch.bernoulli(p_h_given_v)
            
            # update parameters after k-iterations of Gibbs sampling
            delta_W += learning_rate * (torch.ger(p_h_given_v_0, v_0) - torch.ger(p_h_given_v, v_new)) / batch_size
            delta_a += learning_rate * (p_h_given_v_0 - p_h_given_v) / batch_size
            delta_b += learning_rate * (v_0 - v_new) / batch_size
            
        self.W += delta_W
        self.a += delta_a
        self.b += delta_b

In [49]:
# make a training data set
nRow = 3
nCol = 3
data_size = 100
data_length = nRow*nCol

D = Data_for_train(data_size, nRow, nCol)

In [50]:
# initiate RBM parameters
nv = data_length
nh = 10
rbm = RBM(nv, nh)

In [51]:
# Train the RBM
num_epoch = 10
batch_size = 10
learning_rate = 1e-1

for epoch in range(0, num_epoch + 1):
    if epoch > 0:
        for batch_idx in range(int(data_size / batch_size)):
            Batch = D[batch_idx * batch_size : (batch_idx + 1) * batch_size]
            rbm.update_CD(Batch, learning_rate)